In [1]:
import pandas as pd
import numpy

#   To be cleaned:
###      offender_data, offense_data, victim_data
- steps:
    - check value counts for each. Drop low value counts
    - rename variables of interest in excel using codebook (quicker than writing custom rename functions)
    - drop low-percentage variables. even if i have enough rows with a low-% variable to run a decent regression, there might be some underlying reason why the % is low, and a representation skew would throw off the model greatly.
    - check length of each. inner join on case_number?
        - it would make for a more interesting model, i think. see how victim and offender variables interact.
    - subsample? probably. reduce data points from millions to thousands. can always plug in more data to completed model after.
    - decode entries to actual strings. this is tedious, but necessary for model design + interpretation



In [5]:
crimes = pd.read_csv('offense_data.tsv', sep='\t')
perps = pd.read_csv('offender_data.csv')
vics = pd.read_csv('victim_data.csv')

In [6]:
print(f'crimes: {len(crimes)}  perps: {len(perps)}  vics: {len(vics)}')

crimes: 6063523  perps: 65534  vics: 65534


ah, a problem. Numbers is limited to 65,534 rows, so exporting it straight to a CSV seems to have limited my data significantly.
fixed it. have to edit the .tsv originals, and open them here directly (rather than exporting as CSV, which cuts it down to numbers max default). len() takes 8 seconds to calculate 6 million rows.
let's import the new tsvs


In [49]:
crimes = pd.read_csv('offense_data.tsv', sep='\t')
perps = pd.read_csv('offender_data.tsv',sep='\t')
vics = pd.read_csv('victim_data.tsv',sep='\t')

In [10]:
print(f'crimes: {len(crimes)}   perps: {len(perps)}   vics: {len(vics)}')
# they're about the same length, give or take 70,000

crimes: 6063523   perps: 6105830   vics: 6034725


### Sample to reduce load

In [50]:
crimes_full = crimes
perps_full = perps
vics_full = vics

In [30]:
crimes = crimes.sample(5000,random_state=1)
perps = perps.sample(5000,random_state=1)
vics = vics.sample(5000,random_state=1)

In [31]:
crimes.head(10)

,V2001,V2002,V2003,V2004,V2005,V2006,V2007,V2008,V2009,V2010,...,V2016,V2017,V2018,V2019,V2020,V2020B,V2020C,V2020D,V2020E,V2021
1715977,2,20,MA0146000,2W1I0U73LLKR,20160208,120,C,N,,,...,,11,,,88,,,,,1
4493722,2,41,TNMPD0000,BJ0Q3926XEMF,20160916,13B,C,N,,,...,,40,,,88,,,,,1
3752840,2,39,SC0260600,8301-ABM72 N,20161125,290,C,N,,,...,,,,,88,,,,,
951544,2,11,ID0010100,2W1I0URZ72JO,20161208,35A,C,N,,,...,,,,,88,,,,,1
839681,2,14,IA0570100,2W1I0UO ZD1A,20160104,23H,C,N,,,...,,,,,88,,,,,
2473374,2,26,NB0770200,CZ1235HQHKV6,20161229,26B,C,N,,,...,,,,,88,,,,,1
3782956,2,39,SC0310000,1X-HL0706Y1A,20160417,13B,C,N,,,...,,40,,,88,,,,,1
5234761,2,45,VA1120000,5F1Z-090WKBS,20160509,35A,C,N,,,...,,,,,88,,,,,2
5790422,2,46,WASPD0000,7--EY R6MU72,20161107,13A,C,N,,,...,,20,,,88,,,,,1
5491260,2,46,WA0170700,830BKF5RV2 N,20160923,280,C,D,,,...,,,,,88,,,,,2


### Check value_counts for each dataframe, remove low ones
#### build list: vars_of_interest

In [22]:
# crimes.info()
# they're all 5000, even though there's plenty of nulls when i open it in numbers

oh, turns out they don't use nulls, they use one space.
so we need to replace entries = ' ' with entries = None

In [33]:
crimes = crimes.replace(' ', None)
crimes = crimes.replace('', None)

In [34]:
crimes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1715977 to 1139937
Data columns (total 25 columns):
V2001     5000 non-null int64
V2002     5000 non-null int64
V2003     5000 non-null object
V2004     5000 non-null object
V2005     5000 non-null int64
V2006     5000 non-null object
V2007     5000 non-null object
V2008     5000 non-null object
V2009     5000 non-null object
V2010     5000 non-null object
V2011     5000 non-null int64
V2012     5000 non-null object
V2013     5000 non-null object
V2014     5000 non-null object
V2015     5000 non-null object
V2016     5000 non-null object
V2017     5000 non-null object
V2018     5000 non-null object
V2019     5000 non-null object
V2020     5000 non-null int64
V2020B    5000 non-null object
V2020C    5000 non-null object
V2020D    5000 non-null object
V2020E    5000 non-null object
V2021     5000 non-null object
dtypes: int64(5), object(20)
memory usage: 1015.6+ KB


In [41]:
perps.head(10)

,V5001,V5002,V5003,V5004,V5005,V5006,V5007,V5008,V5009,V5010,V5011
1404057,5,16,KY1110000,6S23X6RAPU72,20160130,0,,,,1,
5517245,5,46,WA0150100,7--QYI06MU72,20160906,0,,,,1,
933591,5,11,ID0010000,2W1U0U72S2T4,20150623,1,30,M,W,1,
5226176,5,45,VA0990000,1I1N1MMP77 I,20161219,0,,,,1,
4715279,5,42,TX2201200,830B1OTD-2 N,20160730,2,16,M,B,,N
5920949,5,48,WI0360100,2W1I0U728NBN,20160102,3,23,M,W,,N
1752131,5,21,MI1337900,W10ZUQX IY40,20160109,1,31,M,W,2,
904137,5,14,IA0820200,2W1I0U7PIN24,20160606,1,15,F,W,1,
861868,5,14,IA0770300,2W1I0U7UUX8A,20160327,0,,,,1,
1891,5,1,AL0011200,2W1I0U7PZYJW,20160412,0,,,,1,


In [45]:
perps = perps.applymap(lambda x:None if x == " " else x)

In [46]:
perps.info() # ah, that took care of it

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1404057 to 1437483
Data columns (total 11 columns):
V5001    5000 non-null int64
V5002    5000 non-null int64
V5003    5000 non-null object
V5004    5000 non-null object
V5005    5000 non-null int64
V5006    5000 non-null int64
V5007    3566 non-null object
V5008    3566 non-null object
V5009    3566 non-null object
V5010    4407 non-null object
V5011    1255 non-null object
dtypes: int64(4), object(7)
memory usage: 468.8+ KB


### Dealing with ' ' whitespace entries:
i'm tempted to put that lambda function up top, but that would ruin the 'timeline' of this notebook, so i'll just clean the original .TSVs and re-save them.

In [53]:
perps_full = perps_full.applymap(lambda x:None if x == " " else x)
crimes_full = crimes_full.applymap(lambda x:None if x == " " else x)
vics_full = vics_full.applymap(lambda x:None if x == " " else x)
perps_full.to_csv('perps.csv')
crimes_full.to_csv('crimes.csv')
vics_full.to_csv('vics.csv')
print('done cleaning nulls')

done cleaning nulls


In [54]:
len(perps_full)

6105830

In [ ]:
# we're gonna go by 'crimes', 'perps' and 'vics' for now

In [55]:
crimes = crimes_full.sample(5000,random_state=1) # let's sample down and reduce load 
perps = perps_full.sample(5000,random_state=1)
vics = vics_full.sample(5000,random_state=1)

In [56]:
crimes.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1715977 to 1139937
Data columns (total 25 columns):
V2001     5000 non-null int64
V2002     5000 non-null int64
V2003     5000 non-null object
V2004     5000 non-null object
V2005     5000 non-null int64
V2006     5000 non-null object
V2007     5000 non-null object
V2008     5000 non-null object
V2009     35 non-null object
V2010     0 non-null object
V2011     5000 non-null int64
V2012     369 non-null object
V2013     369 non-null object
V2014     1491 non-null object
V2015     80 non-null object
V2016     16 non-null object
V2017     981 non-null object
V2018     24 non-null object
V2019     0 non-null object
V2020     5000 non-null int64
V2020B    0 non-null object
V2020C    0 non-null object
V2020D    0 non-null object
V2020E    0 non-null object
V2021     4401 non-null object
dtypes: int64(5), object(20)
memory usage: 1015.6+ KB


## Variable selection based on null counts
find the columns with enough values to be meaningful, make lists of their variables


In [58]:
crime_vars = list(crimes) # to be iterated through
perp_vars = list(perps)
vic_vars = list(vics)

In [60]:
crimes['V2012'].isna().sum() # get count of nulls

4631

In [61]:
print(crime_vars)

['V2001', 'V2002', 'V2003', 'V2004', 'V2005', 'V2006', 'V2007', 'V2008', 'V2009', 'V2010', 'V2011', 'V2012', 'V2013', 'V2014', 'V2015', 'V2016', 'V2017', 'V2018', 'V2019', 'V2020', 'V2020B', 'V2020C', 'V2020D', 'V2020E', 'V2021']


In [ ]:
# for col in crime_vars:
#     if crimes.col.isna().sum() > 4500
# wait, nevermind. this is bad.

### We might not want to throw out nulls before we translate variable names
I had a look at the columns again, and some are essentially the same thing separted into 2 or more columns. If the criminal was using some substance or technology or weapon, for instance, is like 5 columns. A NaN indicates they weren't... using anything? or that there was no way to be sure? I'll look that up in the codebook.

/* I stopped here to wonder for a moment: Is it alright that I'm pushing these data files to a public repo, after UMich made me click "Agree" to all those scary legal terms?
And then I thought: no wait, they're available for anyone to click on at (https://crime-data-explorer.fr.cloud.gov/downloads-and-docs). It appears I have been made a fool of. */


Now I'll take a moment to look up each variable name in the codebook and make a list of strings. The least-typing way is probably to write out the lists and then zip to dict.



In [91]:
lst = list(map(lambda x: 'V'+str(x), range(2001,2022)))
lst # cool, but we have '2020B' C D E for extra BIAS MOTIVATIONs
# so let's append those in positionally

['V2001',
 'V2002',
 'V2003',
 'V2004',
 'V2005',
 'V2006',
 'V2007',
 'V2008',
 'V2009',
 'V2010',
 'V2011',
 'V2012',
 'V2013',
 'V2014',
 'V2015',
 'V2016',
 'V2017',
 'V2018',
 'V2019',
 'V2020',
 'V2021']

In [67]:
lst.index('V2020')

19

In [72]:
# we could just type out B, C, D and E, but let's have some fun
ord('B') # so we want 66, 67, 68, 69

66

In [79]:
letters = list(map(lambda x: chr(x), range(66,70)))
letters # cute

['B', 'C', 'D', 'E']

In [80]:
to_ins = ['V2020'] * 4
to_ins

['V2020', 'V2020', 'V2020', 'V2020']

In [89]:
out = []
for base, c in zip(to_ins, letters):
    out.append(base+c)
out

['V2020B', 'V2020C', 'V2020D', 'V2020E']

In [86]:
# lst.insert(19,out)
# lst # nope, this adds the entire list at index
# # list.extend() would just add it to the end.

In [92]:
lst[20:20] = out # this one's interesting. apparently this is the only way
# to .extend at index in list. wonder why this works.
lst

['V2001',
 'V2002',
 'V2003',
 'V2004',
 'V2005',
 'V2006',
 'V2007',
 'V2008',
 'V2009',
 'V2010',
 'V2011',
 'V2012',
 'V2013',
 'V2014',
 'V2015',
 'V2016',
 'V2017',
 'V2018',
 'V2019',
 'V2020',
 'V2020B',
 'V2020C',
 'V2020D',
 'V2020E',
 'V2021']

### crime_vars header list obtained programatically
this took more time than typing them out, but what's important is that the computer did the work

now we've got to type the variable names out ourselves

i'm probably going to continue writing in first-person-plural without capitals or punctuation, for reasons i don't fully understand

some notes on crimes variables:

DAC has three columns, because they only fill one letter per column for Drugs, Alcohol or Computer equipment, flags for having used them during the crime. I should consolidate this into dummy binaries, one for each column.

V2014-16 = 'Type of criminal activity' with multiple slots. For specific offenses (subtypes of 'criminal activity, i assume') these columns specify flags for things like 'Gang Involvement' and 'Using/Consuming'. 
Coding these things in using .apply functions is going to take a *while*, so for this project i'll end up carefully selecting predictors.

Then again, Lasso doesn't care what the entries are called, so long as they match each other. So i'll fret about decoding entries after that.

In [110]:
crimes_cols = [
    'segment_level', 'state', 'agency', 'case_number',
    'date', 'UCR_code', 'attempt_or_complete', 'DAC_1', # drugs, alcohol, or computer equipment used during crime
    'DAC_2', 'DAC_2', 'location_type', 'num_premises_entered',
    'method_entry', 'activity_type_1', 'activity_type_2',
    'activity_type_3', 'weapon_type_1', 'weapon_type_2',
    'weapon_type_3', 'bias_1', 'bias_2', 'bias_3', 'bias_4',
    'bias_5', 'num_common_ORIs'
]

In [96]:
crimes.columns

Index(['V2001', 'V2002', 'V2003', 'V2004', 'V2005', 'V2006', 'V2007', 'V2008',
       'V2009', 'V2010', 'V2011', 'V2012', 'V2013', 'V2014', 'V2015', 'V2016',
       'V2017', 'V2018', 'V2019', 'V2020', 'V2020B', 'V2020C', 'V2020D',
       'V2020E', 'V2021'],
      dtype='object')

In [98]:
c_test = crimes
c_test.columns = col_names # renaming columns is easy if you make a list with an entry for every name

In [99]:
c_test.head(10) # nice, looks like the column names fit

,segment_level,state,agency,case_number,date,UCR_code,attempt_or_complete,DAC_1,DAC_2,DAC_2,...,activity_type_3,weapon_type_1,weapon_type_2,weapon_type_3,bias_1,bias_2,bias_3,bias_4,bias_5,num_common_ORIs
1715977,2,20,MA0146000,2W1I0U73LLKR,20160208,120,C,N,None,None,...,None,11,None,None,88,None,None,None,None,1
4493722,2,41,TNMPD0000,BJ0Q3926XEMF,20160916,13B,C,N,None,None,...,None,40,None,None,88,None,None,None,None,1
3752840,2,39,SC0260600,8301-ABM72 N,20161125,290,C,N,None,None,...,None,None,None,None,88,None,None,None,None,None
951544,2,11,ID0010100,2W1I0URZ72JO,20161208,35A,C,N,None,None,...,None,None,None,None,88,None,None,None,None,1
839681,2,14,IA0570100,2W1I0UO ZD1A,20160104,23H,C,N,None,None,...,None,None,None,None,88,None,None,None,None,None
2473374,2,26,NB0770200,CZ1235HQHKV6,20161229,26B,C,N,None,None,...,None,None,None,None,88,None,None,None,None,1
3782956,2,39,SC0310000,1X-HL0706Y1A,20160417,13B,C,N,None,None,...,None,40,None,None,88,None,None,None,None,1
5234761,2,45,VA1120000,5F1Z-090WKBS,20160509,35A,C,N,None,None,...,None,None,None,None,88,None,None,None,None,2
5790422,2,46,WASPD0000,7--EY R6MU72,20161107,13A,C,N,None,None,...,None,20,None,None,88,None,None,None,None,1
5491260,2,46,WA0170700,830BKF5RV2 N,20160923,280,C,D,None,None,...,None,None,None,None,88,None,None,None,None,2


In [103]:
len(crimes_full)

6063523

In [106]:
len(crimes_full.V2006.value_counts())

52

#### Cleaning note: we're doing all these operations on the n=5000 subsets
#### but we can just remove the subset up top when we want to run the full 6 million

In [100]:
 # let's make lists for the other 2 dataframes
vics.head()

,V4001,V4002,V4003,V4004,V4005,V4006,V4007,V4008,V4009,V4010,...,V4042,V4043,V4044,V4045,V4046,V4047,V4048,V4049,V4050,V4051
5754543,4,46,WASPD0000,3S-XXKI2OF39,20160728,1,23F,None,None,None,...,None,None,None,None,None,None,None,None,None,1
2983587,4,34,OHCLP0000,2W1I0U7NUX0,20160221,1,240,None,None,None,...,None,None,None,None,None,None,None,None,None,1
3524650,4,39,SC0040100,FV-YJ4K7Q-F3,20160806,1,35A,35B,None,None,...,None,None,None,None,None,None,None,None,None,1
4013782,4,41,TN0190100,2W1I05RNS4 A,20160418,1,26B,None,None,None,...,None,None,None,None,None,None,None,None,None,1
173811,4,3,AR0600300,2W1I02PV7X1A,20160721,1,35A,35B,520,None,...,None,None,None,None,None,None,None,None,None,1


In [119]:
len(vics_cols)

50

In [120]:
vics_cols = [
    'segment_level', 'state', 'agency', 'case_number',
    'date', 'victim_seq_num', 'ucr_1', 'ucr_2', 'ucr_3',
    'ucr_4', 'ucr_5', 'ucr_6', 'ucr_7', 'ucr_8', 'ucr_9',
    'ucr_10', 'vic_type', 'officer_activity_type',
    'officer_assignment_type', 'officer_other_juris',
    'vic_age', 'vic_sex', 'vic_race', 'vic_ethn',
    'vic_resident_status', 'assault_homic_1', 'assault_homic_2',
    'add_homic_circumstances', 'injury_1', 'injury_2', 'injury_3',
    'injury_4', 'injury_5',
    'offndr_1_rltn', 'vic_offndr_rltn_1', 'offndr_2_rltn', 'vic_offndr_rltn_2',
    'offndr_3_rltn', 'vic_offndr_rltn_3', 'offndr_4_rltn', 'vic_offndr_rltn_4',
    'offndr_5_rltn', 'vic_offndr_rltn_5', 'offndr_6_rltn', 'vic_offndr_rltn_6',
    'offndr_7_rltn', 'vic_offndr_rltn_7', 'offndr_8_rltn', 'vic_offndr_rltn_8',
    'offndr_9_rltn', 'vic_offndr_rltn_9', 'offndr_10_rltn', 'vic_offndr_rltn_10',
    'num_ori_matches',
]
perps_cols = [
    'segment_level', 'state', 'agency', 'case_number', 'date',
    'perp_seq_num', 'perp_age', 'perp_sex', 'perp_race',
    'num_ori_matches', 'perp_ethn'
]

In [112]:
perps.head()

,V5001,V5002,V5003,V5004,V5005,V5006,V5007,V5008,V5009,V5010,V5011
1404057,5,16,KY1110000,6S23X6RAPU72,20160130,0,None,None,None,1,None
5517245,5,46,WA0150100,7--QYI06MU72,20160906,0,None,None,None,1,None
933591,5,11,ID0010000,2W1U0U72S2T4,20150623,1,30,M,W,1,None
5226176,5,45,VA0990000,1I1N1MMP77 I,20161219,0,None,None,None,1,None
4715279,5,42,TX2201200,830B1OTD-2 N,20160730,2,16,M,B,None,N


In [115]:
len(vics.columns)

54

In [121]:
crimes.columns = crimes_cols
perps.columns = perps_cols
vics.columns = vics_cols # rename columns

### Checkpoint for today. Column names are fixed
- to consider:
    - for ML purposes, we only need to translate the Y variable's entries
        - the model can run perfectly fine on coded data so long as the codes match: it'll run in french, too, so to speak
    - for potential Ys with lots of subcategories, consider binning to reduce num classifiers. quicker computation? better performance
    - what should our Y be?
        - what are people interested in predicting?
            - type of crime
                - if violent or not
            - type of bias
                - if bias or not
            - outcome related variables of crime?
            